# Data-Oriented Programming Paradigms 2024W: Exercise 2
# Group: 31

### Questions we plan to answer:
1. What features can be used to predict whether a hotel or Airbnb will be successful? (For example, why other hotels are more popular than mine)

Here we can use features of neighbourhood (density of POIs, average price of accommodations, public transport availability) or features of the accommodation (prices, breakfast availability, etc)

2. How can we predict the revenue potential of an Airbnb listing in Vienna based on its location relative to public transport, POIs, and the overall tourism trend in the city? 

3. If I plan to open one more accommodation, based on current Airbnb and hotel performance data, can we predict which neighborhoods in Vienna will see a rise in Airbnb demand in the future?


### Datasets we plan to use:
Transport:
https://www.data.gv.at/katalog/dataset/522d3045-0b37-48d0-b868-57c99726b1c4#resources

Airbnb:
https://insideairbnb.com/get-the-data/

Overall tourism:
https://ec.europa.eu/eurostat/web/tourism/database

POI:
https://download.geofabrik.de/europe.html


## Load Data

### installing packages

In [11]:
!pip install geopandas requests

In [2]:
pip install pyproj


Note: you may need to restart the kernel to use updated packages.


In [148]:
pip install fiona

### importing packages

In [1]:
import pandas as pd
import os
import requests
import scipy
import numpy as np
import requests


#### downloading geofabrik's data for closest facilites

In [ ]:
url = "https://download.geofabrik.de/europe/austria-latest-free.shp.zip"
response = requests.get(url)
with open("austria-latest-free.shp.zip", "wb") as file:
    file.write(response.content)

import zipfile

with zipfile.ZipFile("austria-latest-free.shp.zip", "r") as zip_ref:
    zip_ref.extractall("austria_shapefiles")

In [3]:
import geopandas as gpd

pois = gpd.read_file("austria_shapefiles/gis_osm_pois_free_1.shp")

print(pois.head())

     osm_id  code        fclass           name                   geometry
0  15079895  2006     telephone           None  POINT (16.28689 48.19691)
1  15079903  2501   supermarket       Eurospar  POINT (16.28767 48.19697)
2  15080180  2501   supermarket     Billa Plus  POINT (16.29891 48.19776)
3  15080251  2522   sports_shop  Sports Direct  POINT (16.28276 48.19288)
4  17310328  2701  tourist_info           None  POINT (13.49914 47.59028)


#### downloading wienerlinien data for transportation

In [5]:
#function for pulling the data from http request
from io import StringIO
def fetch_csv_from_http(url):
    response = requests.get(url)
    if response.status_code == 200:
        csv_data = StringIO(response.text)
    else:
        return "fetch failed"
    return csv_data


In [7]:
import pandas as pd

# Load the datasets

fahrwegverlaeufe_url = "http://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-fahrwegverlaeufe.csv"
fahrwegverlaeufe_data = fetch_csv_from_http(fahrwegverlaeufe_url)
fahrwegverlaeufe = pd.read_csv(fahrwegverlaeufe_data, sep=';')

gps_punkte_url = "https://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-gps-punkte.csv"
gps_punkte_data = fetch_csv_from_http(gps_punkte_url)
gps_punkte = pd.read_csv(gps_punkte_data, sep=';')

haltepunkte_url = "https://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-haltepunkte.csv"
haltepunkte_data = fetch_csv_from_http(haltepunkte_url)
haltepunkte = pd.read_csv(haltepunkte_data, sep=';')

haltestellen_url = "https://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-haltestellen.csv"
haltestellen_data = fetch_csv_from_http(haltestellen_url)
haltestellen = pd.read_csv(haltestellen_data, sep=';')

linien_url = "https://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-linien.csv"
linien_data = fetch_csv_from_http(linien_url)
linien = pd.read_csv(linien_data, sep=';')

steige_url = "https://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-steige.csv"
steige_data = fetch_csv_from_http(steige_url)
steige = pd.read_csv(steige_data, sep=';')

verbindungen_url = "https://www.wienerlinien.at/ogd_realtime/doku/ogd/wienerlinien-ogd-verbindungen.csv"
verbindungen_data = fetch_csv_from_http(verbindungen_url)
verbindungen = pd.read_csv(verbindungen_data, sep=';')



In [8]:
gps_punkte.head(10)

,FromStartStop,ToEndStop,TransportMode,SeqCount,Longitude,Latitude,DistanceFromStartPoint
0,59,65,1,1,16.417250,48.230871,5
1,59,65,1,2,16.416807,48.231183,46
2,59,65,1,3,16.416320,48.231542,93
3,59,65,1,4,16.415859,48.231881,137
4,59,65,1,5,16.415348,48.232243,185
5,59,65,1,6,16.415215,48.232342,198
6,59,65,1,7,16.415210,48.232410,204
7,59,65,1,8,16.415218,48.232466,210
8,59,65,1,9,16.415257,48.232490,213
9,59,65,1,10,16.415450,48.232580,228


In [11]:
haltepunkte.head(10)

,StopID,DIVA,StopText,Municipality,MunicipalityID,Longitude,Latitude
0,3,NaN,Messe Prater,NaN,NaN,NaN,NaN
1,5,60200195.0,BÃ¶rse,Wien,49000001.0,16.366002,48.215563
2,7,60201848.0,"Praterstern, LassallestraÃe",Wien,49000001.0,16.393769,48.219549
3,8,60201349.0,Hauptbahnhof S U,Wien,49000001.0,16.374976,48.185919
4,9,60200619.0,Julius-Raab-Platz,Wien,49000001.0,16.382567,48.210343
5,10,60200245.0,"Stubentor U, Dr.-Karl-Lueger-Platz",Wien,49000001.0,16.379890,48.206727
6,11,60201460.0,Weihburggasse,Wien,49000001.0,16.377752,48.204608
7,12,60201193.0,Schwarzenbergplatz,Wien,49000001.0,16.374617,48.201818
8,13,60200975.0,"Oper, Karlsplatz U",Wien,49000001.0,16.370089,48.202332
9,14,60200190.0,Burgring,Wien,49000001.0,16.363720,48.203861


In [13]:
haltestellen.head(10)

,DIVA,PlatformText,Municipality,MunicipalityID,Longitude,Latitude
0,60200001,Schrankenberggasse,Wien,49000001,48.173801,16.389807
1,60200002,Achengasse,Wien,49000001,48.284460,16.448925
2,60200003,Ada-Christen-Gasse,Wien,49000001,48.152830,16.386043
3,60200004,Adam-Betz-Gasse,Wien,49000001,48.215581,16.535469
4,60200005,Adamovichgasse,Wien,49000001,48.142664,16.338621
5,60200006,Adolf-Loos-Gasse,Wien,49000001,48.277394,16.454199
6,60200007,Adolf-Unger-Gasse,Wien,49000001,48.148676,16.385882
7,60200008,Afritschgasse,Wien,49000001,48.247098,16.451692
8,60200009,Agavenweg,Wien,49000001,48.251100,16.506480
9,60200010,"Neustift, Agnesgasse",Wien,49000001,48.250448,16.307567


In [15]:
linien.head(10)

,LineID,LineText,SortingHelp,Realtime,MeansOfTransport
0,301,U1,1,0,ptMetro
1,302,U2,2,0,ptMetro
2,303,U3,3,0,ptMetro
3,304,U4,4,0,ptMetro
4,306,U6,6,0,ptMetro
5,101,1,101,1,ptTram
6,102,2,102,1,ptTram
7,105,5,105,1,ptTram
8,106,6,106,1,ptTram
9,109,9,109,1,ptTram


In [17]:
path_airbnb = "Data/Airbnb/"

calendar_path = os.path.join(path_airbnb, "calendar_detailed.csv")
listings_path = os.path.join(path_airbnb, "listings_detailed.csv")
neighbourhoods_path = os.path.join(path_airbnb, "neighbourhoods.csv")
reviews_path = os.path.join(path_airbnb, "reviews_detailed.csv")

calendar_df = pd.read_csv(calendar_path)
listings_df = pd.read_csv(listings_path)
neighbourhoods_df = pd.read_csv(neighbourhoods_path)
reviews_df = pd.read_csv(reviews_path)

path_transport = "Data/Transport/"

stops_path = os.path.join(path_transport, "OEFFHALTESTOGD.csv")
lines_path = os.path.join(path_transport, "OEFFLINIENOGD.csv")

stops_df = pd.read_csv(stops_path)
lines_df = pd.read_csv(lines_path)

path_tourism = "Data/Tourism/"

tour_occ_arm_path = os.path.join(path_tourism, "estat_tour_occ_arm.tsv")
tour_occ_mnor_path = os.path.join(path_tourism, "estat_tour_occ_mnor.tsv")
tour_occ_nim_path = os.path.join(path_tourism, "estat_tour_occ_nim.tsv")

tour_occ_arm_df = pd.read_csv(tour_occ_arm_path, sep='\t')
tour_occ_mnor_df = pd.read_csv(tour_occ_mnor_path, sep='\t')
tour_occ_nim_df = pd.read_csv(tour_occ_nim_path, sep='\t')

In [18]:
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,275668,2024-09-11,f,$59.00,NaN,2,1125
1,275668,2024-09-12,f,$59.00,NaN,2,1125
2,275668,2024-09-13,f,$59.00,NaN,2,1125
3,275668,2024-09-14,f,$59.00,NaN,2,1125
4,275668,2024-09-15,f,$59.00,NaN,2,1125


In [19]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,38768,https://www.airbnb.com/rooms/38768,20240911015603,2024-09-11,city scrape,central cityapartement- wifi- nice neighbourhood,39m² apartment with beautiful courtyard of the...,the Karmeliterviertel became very popular in t...,https://a0.muscache.com/pictures/ad4089a3-5355...,166283,...,4.94,4.77,4.69,NaN,t,3,3,0,0,2.38
1,40625,https://www.airbnb.com/rooms/40625,20240911015603,2024-09-11,city scrape,"Near Palace Schönbrunn, Apt. 1",Welcome to my Apt. 1!<br /><br />This is a 2be...,The neighbourhood offers plenty of restaurants...,https://a0.muscache.com/pictures/11509144/d55c...,175131,...,4.94,4.61,4.72,NaN,t,14,13,1,0,1.26
2,51287,https://www.airbnb.com/rooms/51287,20240911015603,2024-09-11,city scrape,little studio- next to citycenter- wifi- nice ...,small studio in new renovated old house and ve...,The neighbourhood has a lot of very nice littl...,https://a0.muscache.com/pictures/25163038/1c4e...,166283,...,4.95,4.87,4.59,NaN,t,3,3,0,0,2.29
3,70637,https://www.airbnb.com/rooms/70637,20240911015603,2024-09-11,city scrape,Flat in the Center with Terrace,NaN,NaN,https://a0.muscache.com/pictures/925691/c8c1bd...,358842,...,4.76,4.81,4.72,NaN,f,3,1,2,0,0.71
4,78416,https://www.airbnb.com/rooms/78416,20240911015603,2024-09-11,city scrape,Nice Apartment in Vienna center,"Newly renovated modern, comfortable apartment ...",Stadthalle – Viennas largest event center is j...,https://a0.muscache.com/pictures/7b8ed252-4016...,421075,...,4.43,4.41,4.47,NaN,t,2,2,0,0,1.09


In [20]:
neighbourhoods_df.head()

,neighbourhood_group,neighbourhood
0,NaN,Alsergrund
1,NaN,Brigittenau
2,NaN,Dbling
3,NaN,Donaustadt
4,NaN,Favoriten


## Merging files

In [22]:
combined_df = pd.merge(calendar_df, listings_df, how='inner', left_on='listing_id', right_on='id')

Calculate nearest points between accomodation and POI

In [24]:
from sklearn.neighbors import KDTree
import numpy as np
import pandas as pd


In [25]:
# Coordinates combined_df and pois
combined_coords = combined_df[['longitude', 'latitude']].values
poi_coords = np.array([p.coords[0] for p in pois.geometry])

In [26]:
unique_combined_coords = np.unique(combined_coords, axis=0)

In [27]:
# Creating KDTree for POI
tree = KDTree(poi_coords, metric='euclidean')

# Search radius (in degrees; for example ~1 km if latitude/longitude in degrees)
radius = 0.01

# Find all nearest points for each coordinate from combined_df
indices_within_radius = tree.query_radius(unique_combined_coords, r=radius)

In [28]:
coord_to_poi = {}
for coord, indices in zip(unique_combined_coords, indices_within_radius):
    # Store osm_id of nearest POI for each unique coordinate
    coord_to_poi[tuple(coord)] = pois.iloc[indices].osm_id.tolist()

In [29]:
# map closest POIs to each row in combined_df
def map_pois(row):
    coord = (row['longitude'], row['latitude'])
    return coord_to_poi.get(coord, [])

In [30]:
combined_df['nearest_osm_ids'] = combined_df.apply(map_pois, axis=1)

In [31]:
combined_df

,listing_id,date,available,price_x,adjusted_price,minimum_nights_x,maximum_nights_x,id,listing_url,scrape_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,nearest_osm_ids
0,275668,2024-09-11,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
1,275668,2024-09-12,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
2,275668,2024-09-13,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
3,275668,2024-09-14,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
4,275668,2024-09-15,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5254471,1242124355670149881,2025-09-06,f,$999.00,NaN,5,179,1242124355670149881,https://www.airbnb.com/rooms/1242124355670149881,20240911015603,...,NaN,NaN,NaN,t,134,48,85,1,NaN,"[8745572810, 9571732563, 9571732565, 858608072..."
5254472,1242124355670149881,2025-09-07,f,$999.00,NaN,5,179,1242124355670149881,https://www.airbnb.com/rooms/1242124355670149881,20240911015603,...,NaN,NaN,NaN,t,134,48,85,1,NaN,"[8745572810, 9571732563, 9571732565, 858608072..."
5254473,1242124355670149881,2025-09-08,f,$999.00,NaN,5,179,1242124355670149881,https://www.airbnb.com/rooms/1242124355670149881,20240911015603,...,NaN,NaN,NaN,t,134,48,85,1,NaN,"[8745572810, 9571732563, 9571732565, 858608072..."
5254474,1242124355670149881,2025-09-09,f,$999.00,NaN,5,179,1242124355670149881,https://www.airbnb.com/rooms/1242124355670149881,20240911015603,...,NaN,NaN,NaN,t,134,48,85,1,NaN,"[8745572810, 9571732563, 9571732565, 858608072..."


In [36]:
combined_df.describe()

,listing_id,adjusted_price,minimum_nights_x,maximum_nights_x,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,5.254476e+06,0.0,5.254476e+06,5.254476e+06,5.254476e+06,5.254476e+06,5.254476e+06,5.254111e+06,5.254111e+06,0.0,...,4.371950e+06,4.371950e+06,4.371950e+06,4.371585e+06,0.0,5.254476e+06,5.254476e+06,5.254476e+06,5.254476e+06,4.371950e+06
mean,5.007423e+17,NaN,8.716305e+00,5.973711e+05,5.007423e+17,2.024091e+13,2.172167e+08,3.368401e+01,4.595286e+01,NaN,...,4.806941e+00,4.787986e+00,4.678215e+00,4.628297e+00,NaN,2.546457e+01,2.337796e+01,2.007701e+00,4.515198e-02,1.587380e+00
std,5.017594e+17,NaN,2.821152e+01,3.579160e+07,5.017594e+17,2.193289e+03,1.996699e+08,1.037440e+02,1.750910e+02,NaN,...,3.903655e-01,4.125226e-01,4.036864e-01,4.580510e-01,NaN,6.274657e+01,6.169070e+01,8.726445e+00,4.474843e-01,1.987450e+00
min,3.876800e+04,NaN,1.000000e+00,1.000000e+00,3.876800e+04,2.024091e+13,5.783000e+03,1.000000e+00,1.000000e+00,NaN,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02
25%,2.722653e+07,NaN,1.000000e+00,1.800000e+02,2.722653e+07,2.024091e+13,3.015884e+07,1.000000e+00,1.000000e+00,NaN,...,4.790000e+00,4.750000e+00,4.560000e+00,4.530000e+00,NaN,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.400000e-01
50%,5.955650e+17,NaN,2.000000e+00,3.650000e+02,5.955650e+17,2.024091e+13,1.485427e+08,4.000000e+00,5.000000e+00,NaN,...,4.910000e+00,4.910000e+00,4.770000e+00,4.740000e+00,NaN,3.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,9.300000e-01
75%,1.004143e+18,NaN,5.000000e+00,1.125000e+03,1.004143e+18,2.024091e+13,4.136264e+08,2.000000e+01,2.300000e+01,NaN,...,5.000000e+00,5.000000e+00,4.940000e+00,4.880000e+00,NaN,1.700000e+01,1.500000e+01,1.000000e+00,0.000000e+00,2.300000e+00
max,1.242124e+18,NaN,1.125000e+03,2.147484e+09,1.242124e+18,2.024091e+13,6.507529e+08,2.981000e+03,7.164000e+03,NaN,...,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,NaN,3.680000e+02,3.680000e+02,8.500000e+01,1.100000e+01,8.418000e+01


In [37]:
combined_df.isnull().sum()

listing_id                                            0
date                                                  0
available                                             0
price_x                                               0
adjusted_price                                  5254476
                                                 ...   
calculated_host_listings_count_entire_homes           0
calculated_host_listings_count_private_rooms          0
calculated_host_listings_count_shared_rooms           0
reviews_per_month                                882526
nearest_osm_ids                                       0
Length: 83, dtype: int64

In [38]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5254476 entries, 0 to 5254475
Data columns (total 83 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   listing_id                                    int64  
 1   date                                          object 
 2   available                                     object 
 3   price_x                                       object 
 4   adjusted_price                                float64
 5   minimum_nights_x                              int64  
 6   maximum_nights_x                              int64  
 7   id                                            int64  
 8   listing_url                                   object 
 9   scrape_id                                     int64  
 10  last_scraped                                  object 
 11  source                                        object 
 12  name                                          object 
 1

In [39]:
combined_df.head()

,listing_id,date,available,price_x,adjusted_price,minimum_nights_x,maximum_nights_x,id,listing_url,scrape_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,nearest_osm_ids
0,275668,2024-09-11,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
1,275668,2024-09-12,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
2,275668,2024-09-13,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
3,275668,2024-09-14,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."
4,275668,2024-09-15,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,4.84,4.73,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146..."


In [40]:
haltestellen_coords = haltestellen[['Longitude', 'Latitude']].values

In [41]:
# Creating KDTree for haltestellen (stops)
tree = KDTree(haltestellen_coords, metric='euclidean')

# Search radius (in degrees; for example ~1 km if latitude/longitude in degrees)
radius = 0.01

# Find all nearest points for each coordinate from combined_df
indices_within_radius = tree.query_radius(unique_combined_coords, r=radius)

In [42]:
coord_to_haltestellen = {}
for coord, indices in zip(unique_combined_coords, indices_within_radius):
    # Store osm_id of nearest POI for each unique coordinate
    coord_to_haltestellen[tuple(coord)] = haltestellen.iloc[indices].DIVA.tolist()

In [43]:
# map closest POIs to each row in combined_df
def map_haltestellen(row):
    coord = (row['Longitude'], row['Latitude'])
    return coord_to_haltestellen.get(coord, [])

In [65]:
combined_df['nearest_stop_ID'] = combined_df.apply(map_pois, axis=1) #I used haltestellen - DIVA

In [67]:
combined_df.head()

,listing_id,date,available,price_x,adjusted_price,minimum_nights_x,maximum_nights_x,id,listing_url,scrape_id,...,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,nearest_osm_ids,nearest_haltestellen_DIVA,nearest_stop_ID
0,275668,2024-09-11,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146..."
1,275668,2024-09-12,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146..."
2,275668,2024-09-13,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146..."
3,275668,2024-09-14,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146..."
4,275668,2024-09-15,f,$59.00,NaN,2,1125,275668,https://www.airbnb.com/rooms/275668,20240911015603,...,NaN,t,3,3,0,0,1.97,"[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146...","[7906474310, 1290993459, 617621809, 1203277146..."
